# Importar Libreria

In [1]:
import os
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Cargar dataset

El dataset que vamos a utilizar es una clasificación realizada por el Departamento de Asuntos Económicos y Sociales de las Naciones Unidas, obtendio del siguiente enlace:https://unstats.un.org/unsd/methodology/m49/overview

En mi proyecto final, uno de los objetivos es analizar la evolución del procentaje de la población con acceso a internet.La idea es estudiar la evolución en las diferentes zonas geográficas. 

Por lo tanto en este notebook obtengo la clasificación mundial de regiones y sub-regiones, que posteriormente usaré en mi estudio.

In [2]:
df_geography= pd.read_csv('UNSD — Methodology.csv',sep=';')

In [3]:
df_geography.shape

(249, 15)

In [4]:
df_geography.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Global Code                              249 non-null    int64  
 1   Global Name                              249 non-null    object 
 2   Region Code                              248 non-null    float64
 3   Region Name                              248 non-null    object 
 4   Sub-region Code                          248 non-null    float64
 5   Sub-region Name                          248 non-null    object 
 6   Intermediate Region Code                 108 non-null    float64
 7   Intermediate Region Name                 108 non-null    object 
 8   Country or Area                          249 non-null    object 
 9   M49 Code                                 249 non-null    int64  
 10  ISO-alpha2 Code                          247 non-n

In [5]:
df_geography.head()

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN


# Creación del sub dataset

In [6]:
# lista de las columnas que nos interesa mantener
col=['Region Name', 'Sub-region Name','Country or Area','ISO-alpha3 Code']

In [7]:
df_geography= df_geography[col]

In [8]:
df_geography.head()

,Region Name,Sub-region Name,Country or Area,ISO-alpha3 Code
0,Africa,Northern Africa,Algeria,DZA
1,Africa,Northern Africa,Egypt,EGY
2,Africa,Northern Africa,Libya,LBY
3,Africa,Northern Africa,Morocco,MAR
4,Africa,Northern Africa,Sudan,SDN


In [9]:
df_geography.duplicated().sum()#comprobamos si hay valores duplicados

0

In [10]:
df_geography.isnull().sum()#comprobamos los valores nulos

Region Name        1
Sub-region Name    1
Country or Area    0
ISO-alpha3 Code    1
dtype: int64

# Añadimos al sub df la columna Grupo de Ingreso del Banco Mundial

Nos interesa conocer para cada país el grupo de ingreso perteneciente(alto,bajo,medio).Utilizamos otro dataset porque en la primera clasifición no teníamos esta infromación. Por otro lado este segundo dataset nos es muy útil porque aparece la columna País, de esta manera descartaremos del dataset df_geography areas que nos son países y que por tanto no nos interesan. <br>
Enlace dataset para grupo de ingresos (clasificación Banco  Mundial): https://ilostat.ilo.org/es/resources/concepts-and-definitions/classification-country-groupings/

In [11]:

df_geo_ingresos= pd.read_csv('countries_es.csv')

In [12]:
df_geo_ingresos.head()

,ISO3 Code,País,Región,Subregión,Subregión detallada,Grupo de Ingresos del Banco Mundial,¿Es miembro de la OIT?,¿Incluido en las estimaciones de la OIT?
0,AFG,Afganist?n,Asia y el Pac?fico,Asia meridonal,Asia meridonal,Ingreso bajo,Si,Si
1,ALB,Albania,Europa y Asia Central,"Europa del Norte, Sur y Occidental",Sur de Europa,Ingreso mediano alto,Si,Si
2,DZA,Argelia,?frica,Norte de ?frica,Norte de ?frica,Ingreso mediano bajo,Si,Si
3,ASM,Samoa Americana,Asia y el Pac?fico,Asia sudoriental y el Pac?fico,Islas del Pac?fico,Ingreso mediano alto,No,No
4,AND,Andorra,Europa y Asia Central,"Europa del Norte, Sur y Occidental",Sur de Europa,Ingreso alto,No,No


In [13]:
df_geo_ingresos.columns

Index(['ISO3 Code', 'País', 'Región', 'Subregión ', 'Subregión detallada',
       'Grupo de Ingresos del Banco Mundial ', '¿Es miembro de la OIT? ',
       '¿Incluido en las estimaciones de la OIT? '],
      dtype='object')

Para juntarlo al dataset df_geography solo necesitaremos la columna de codigo por pais (para el merge) y Grupo de Ingresos que es la que nos interesa añadir

In [14]:
col2=['ISO3 Code','Grupo de Ingresos del Banco Mundial ']

In [15]:
df_geo_ingresos=df_geo_ingresos[col2]

In [16]:
df_geo_ingresos.duplicated().sum()#comprobamos los valores duplicados

0

In [17]:
df_geo_ingresos.isnull().sum().sum()# no hay valores nulos por lo que cada ISO 3 Code es un país

0

In [18]:
df_geo_ingresos.head()

,ISO3 Code,Grupo de Ingresos del Banco Mundial
0,AFG,Ingreso bajo
1,ALB,Ingreso mediano alto
2,DZA,Ingreso mediano bajo
3,ASM,Ingreso mediano alto
4,AND,Ingreso alto


Mergue dataset df_geography df_geo_ingresos

In [19]:
df_geography.columns

Index(['Region Name', 'Sub-region Name', 'Country or Area', 'ISO-alpha3 Code'], dtype='object')

In [20]:
df_geo_ingresos.columns

Index(['ISO3 Code', 'Grupo de Ingresos del Banco Mundial '], dtype='object')

In [21]:
df_geo=pd.merge(df_geography,df_geo_ingresos,left_on='ISO-alpha3 Code',right_on='ISO3 Code',how='left')
df_geo

,Region Name,Sub-region Name,Country or Area,ISO-alpha3 Code,ISO3 Code,Grupo de Ingresos del Banco Mundial
0,Africa,Northern Africa,Algeria,DZA,DZA,Ingreso mediano bajo
1,Africa,Northern Africa,Egypt,EGY,EGY,Ingreso mediano bajo
2,Africa,Northern Africa,Libya,LBY,LBY,Ingreso mediano alto
3,Africa,Northern Africa,Morocco,MAR,MAR,Ingreso mediano bajo
4,Africa,Northern Africa,Sudan,SDN,SDN,Ingreso bajo
...,...,...,...,...,...,...
244,Oceania,Polynesia,Samoa,WSM,WSM,Ingreso mediano bajo
245,Oceania,Polynesia,Tokelau,TKL,TKL,Ingreso mediano bajo
246,Oceania,Polynesia,Tonga,TON,TON,Ingreso mediano alto
247,Oceania,Polynesia,Tuvalu,TUV,TUV,Ingreso mediano alto


In [22]:
df_geo.isnull().sum()

Region Name                              1
Sub-region Name                          1
Country or Area                          0
ISO-alpha3 Code                          1
ISO3 Code                               16
Grupo de Ingresos del Banco Mundial     16
dtype: int64

In [23]:
df_geo.drop('ISO-alpha3 Code',axis=1,inplace=True)#nos quedamos con ISO3 Code ya que corresponde a países 
df_geo

,Region Name,Sub-region Name,Country or Area,ISO3 Code,Grupo de Ingresos del Banco Mundial
0,Africa,Northern Africa,Algeria,DZA,Ingreso mediano bajo
1,Africa,Northern Africa,Egypt,EGY,Ingreso mediano bajo
2,Africa,Northern Africa,Libya,LBY,Ingreso mediano alto
3,Africa,Northern Africa,Morocco,MAR,Ingreso mediano bajo
4,Africa,Northern Africa,Sudan,SDN,Ingreso bajo
...,...,...,...,...,...
244,Oceania,Polynesia,Samoa,WSM,Ingreso mediano bajo
245,Oceania,Polynesia,Tokelau,TKL,Ingreso mediano bajo
246,Oceania,Polynesia,Tonga,TON,Ingreso mediano alto
247,Oceania,Polynesia,Tuvalu,TUV,Ingreso mediano alto


Comprobamos que los valores nulos son de areas que vamos a elimininar

In [24]:
df_geo[df_geo['ISO3 Code'].isna()]

,Region Name,Sub-region Name,Country or Area,ISO3 Code,Grupo de Ingresos del Banco Mundial
7,Africa,Sub-Saharan Africa,British Indian Ocean Territory,NaN,NaN
13,Africa,Sub-Saharan Africa,French Southern Territories,NaN,NaN
65,Americas,Latin America and the Caribbean,"Bonaire, Sint Eustatius and Saba",NaN,NaN
79,Americas,Latin America and the Caribbean,Saint Barthélemy,NaN,NaN
98,Americas,Latin America and the Caribbean,Bouvet Island,NaN,NaN
108,Americas,Latin America and the Caribbean,South Georgia and the South Sandwich Islands,NaN,NaN
117,NaN,NaN,Antarctica,NaN,NaN
178,Europe,Northern Europe,Åland Islands,NaN,NaN
181,Europe,Northern Europe,Sark,NaN,NaN
192,Europe,Northern Europe,Svalbard and Jan Mayen Islands,NaN,NaN


In [25]:
df_geo2=df_geo.copy()

In [26]:
df_geo2.dropna(inplace=True)

In [27]:
df_geo2.duplicated().sum()

0

In [28]:
df_geo2.isnull().sum()


Region Name                             0
Sub-region Name                         0
Country or Area                         0
ISO3 Code                               0
Grupo de Ingresos del Banco Mundial     0
dtype: int64

In [29]:
df_geo2.rename(columns = {'Country or Area':'Country Name'}, inplace = True)

In [30]:
px.scatter(df_geo2, x='ISO3 Code', y='Region Name',color='Sub-region Name',width=1000, height=600,color_discrete_sequence=px.colors.sequential.RdBu)

In [31]:
df_geo2['ISO3 Code'].nunique()

233

In [32]:
var1=df_geo2['Grupo de Ingresos del Banco Mundial '].value_counts()
var1

Ingreso alto            88
Ingreso mediano bajo    59
Ingreso mediano alto    59
Ingreso bajo            27
Name: Grupo de Ingresos del Banco Mundial , dtype: int64

In [33]:
#de acuerdo con el BM, ¿Cuántos países hay en cada nivel de ingresos?
px.bar(x=var1, y= var1.index, title='Grupo de ingresos',color_discrete_sequence=px.colors.sequential.Tealgrn_r)

In [34]:
df_geo2.to_csv('geography.csv', index=False)